In [1]:
# !pip install  "unstructured[all-docs]" chromadb langchain ollama langchain_community

In [2]:
!ollama pull qwen2:7b 
!ollama pull nomic-embed-text

pulling manifest в ™ pulling manifest в № pulling manifest в ё pulling manifest в ј pulling manifest в ј pulling manifest в ¦ pulling manifest в ¦ pulling manifest в ‡ pulling manifest в Џ pulling manifest 
pulling 43f7a214e532... 100% в–•в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–Џ 4.4 GB                         
pulling 62fbfd9ed093... 100% в–•в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–Џ  182 B                         
pulling c156170b718e... 100% в–•в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–Џ  11 KB                         
pulling f02dd72bb242... 100% в–•в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–Џ   59 B                         
pulling 648f809ced2b... 100% в–•в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–€в–Џ  485 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 
pulling manifest в ™ pulling manifest в № pulling manifest в ё pulling manifest в ј pulling manifest в ґ pulling manifest в ¦ pulling

In [3]:
!ollama list

NAME                    	ID          	SIZE  	MODIFIED               
nomic-embed-text:latest 	0a109f422b47	274 MB	Less than a second ago	
qwen2:7b                	e0d4e1163c58	4.4 GB	1 second ago          	
llama3:latest           	365c0bd3c000	4.7 GB	44 hours ago          	
qwen2:7b-instruct-q3_K_M	f27ce4958e87	3.8 GB	45 hours ago          	
qwen2:1.5b              	f6daf2b25194	934 MB	45 hours ago          	
llama2-uncensored:latest	44040b922233	3.8 GB	5 days ago            	
moondream:latest        	55fc3abd3867	1.7 GB	7 days ago            	
phi3:latest             	64c1188f2485	2.4 GB	9 days ago            	


In [4]:
from langchain.document_loaders import DirectoryLoader

dir_path='./data'

def load_documents(data_path):
    loader= DirectoryLoader(data_path, glob='*')
    return loader.load()

documents=load_documents(dir_path)
# print(documents)

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_text(documents:list):
    text_spliter= RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=300,
        length_function=len,
        add_start_index=True
    )

    chunks=text_spliter.split_documents(documents)
    return chunks

chunks=split_text(documents)
# print(chunks) 

In [6]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma


ollama_emb = OllamaEmbeddings(model="nomic-embed-text",show_progress=True)
vectorstore = Chroma.from_documents(documents=chunks, embedding=ollama_emb,persist_directory="./chroma_db")

OllamaEmbeddings: 100%|██████████| 21/21 [00:52<00:00,  2.51s/it]


In [7]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 
from langchain import PromptTemplate
from langchain.chains import RetrievalQA


template = """Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use three sentences maximum and keep the answer as concise as possible.
    {context}
    Question: {question}
    Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template,
    )

llm = Ollama(model="qwen2:7b", callback_manager = CallbackManager([StreamingStdOutCallbackHandler()]))


qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)



In [8]:
while True:
    YOUR_QUESTION = input("\nYour question(/bye for exit): ")
    if YOUR_QUESTION == "/bye":
        break
    if YOUR_QUESTION.strip() == "":
        continue

    result = qa_chain({"query": YOUR_QUESTION })


Your question(/bye for exit):  What is the thirty-first rule of the Internet?


C:\Users\plumb\anaconda3\envs\ollama_agent\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.15s/it]


The thirty-first rule of the Internet is: "Tits or GTFO—the choice is yours."


Your question(/bye for exit):   What is the third rule of the Internet?


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.32s/it]


...

Rule 77 states that "The Internet makes you stupid."


Your question(/bye for exit):  What is the conclusion?


OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.13s/it]


The conclusion from the provided data seems to highlight that there are significant differences in language usage among bloggers based on their ages (10s, 20s, 30s) and genders. The analysis shows variations in the frequency of word classes and specific words used by different age groups and both male and female authors. The researchers have found statistically significant differences in these linguistic patterns.

The true test of significance is then presented as whether these observed differences can be effectively utilized to predict an author's age and gender. This suggests that if there are indeed meaningful variations between the language profiles of bloggers across the specified demographics, it would be possible to develop models or systems that could automatically profile authors based on their written content.

Thus, the conclusion might emphasize the potential for developing automated tools capable of identifying blogging styles related to age and gender through analyzing l


Your question(/bye for exit):  /bye
